In [ ]:
import pandas as pd
import glob
import os
from autogluon.tabular import TabularPredictor

# Path to the directory containing feather files
feather_directory = "combined_data"

# Initialize an empty list to store individual DataFrames with yolomodel
dataframes = []

# Read each feather file
for file in glob.glob(f"{feather_directory}/*.feather"):
    # Extract the base filename
    filename = os.path.basename(file)

    # Extract yolomodel from the filename
    yolomodel = filename.split('_')[1]

    # Read the dataframe
    df = pd.read_feather(file)

    # Remove timestamp column, assuming it's named 'timestamp' as a placeholder
    df = df.drop(columns=['timestamp']) if 'timestamp' in df.columns else df

    # Add the extracted yolomodel as a new column
    df['yolomodel'] = yolomodel

    # Append to the list of DataFrames
    dataframes.append(df)

# Determine common columns across all DataFrames
common_columns = list(set.intersection(*(set(df.columns) for df in dataframes)))

# Standardize each DataFrame to have only columns that are common across all DataFrames
dataframes = [df[common_columns] for df in dataframes]

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_feather("datasets/predict_yolomodel.feather")